In [ ]:
!pip install langchain chromadb google-generativeai tiktoken pypdf langchain-google-genai langchain-core langchain-community

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.schema import Document

# using langchain to create the data using document object

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [4]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [24]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/text-embedding-004")

In [25]:
vector_store = Chroma(
      embedding_function = embeddings,
      persist_directory = 'my_chroma_db',
      collection_name = 'sample'
)

In [26]:
print("model:", embeddings.model)


model: models/text-embedding-004


In [ ]:
# store documents

vector_store.add_documents(docs)

['aa8b6dcc-38c2-444a-8713-86bc1e89f76e',
 'f8459928-49fb-4bfa-becd-acd57cf17061',
 '297f29a4-ed63-4245-9fc4-0d35113a0d6b',
 'a45a1eb5-135e-432f-b46b-fa19da2cad12',
 '77c087a9-e091-4ae9-ad5f-863f42da1717']

In [ ]:
# view documents

vector_store.get(include = ['embeddings', 'documents', 'metadatas'])

{'ids': ['aa8b6dcc-38c2-444a-8713-86bc1e89f76e',
  'f8459928-49fb-4bfa-becd-acd57cf17061',
  '297f29a4-ed63-4245-9fc4-0d35113a0d6b',
  'a45a1eb5-135e-432f-b46b-fa19da2cad12',
  '77c087a9-e091-4ae9-ad5f-863f42da1717'],
 'embeddings': array([[-0.05562957, -0.0203272 , -0.01269856, ...,  0.00051751,
         -0.02111294, -0.01154474],
        [-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.03262324,  0.01831091, -0.0045399 , ...,  0.00915938,
         -0.02804055, -0.03489326],
        [-0.0146971 , -0.01844227, -0.02503126, ...,  0.01035003,
         -0.01682558, -0.01603061],
        [-0.03304848, -0.03280932, -0.02324565, ..., -0.00354889,
         -0.00507506, -0.00217535]], shape=(5, 768)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo

In [29]:
# search in the documents

vector_store.similarity_search(
      query = "who is the best bowler?",
      k = 2
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.')]

In [ ]:
# search with the similarity score

vector_store.similarity_search_with_score(
      query = "who is the fastest bowler",
      k = 1
)

# if the score is less then it means that it is close, if the distance is more then there is more dissimilarity

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.7437963485717773)]

In [32]:
# meta data filtering

vector_store.similarity_search_with_score(
      query = "",
      filter = {"team" : "Chennai Super Kings"}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  1.1392717361450195),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  1.1505862474441528)]

In [33]:
# update document

updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id = "aa8b6dcc-38c2-444a-8713-86bc1e89f76e", document = updated_doc1)

In [35]:
# view document

vector_store.get(include = ['embeddings', 'documents', 'metadatas'])

{'ids': ['aa8b6dcc-38c2-444a-8713-86bc1e89f76e',
  'f8459928-49fb-4bfa-becd-acd57cf17061',
  '297f29a4-ed63-4245-9fc4-0d35113a0d6b',
  'a45a1eb5-135e-432f-b46b-fa19da2cad12',
  '77c087a9-e091-4ae9-ad5f-863f42da1717'],
 'embeddings': array([[-0.03802421, -0.03025424, -0.00024509, ...,  0.01538167,
         -0.02452593, -0.01501566],
        [-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.03262324,  0.01831091, -0.0045399 , ...,  0.00915938,
         -0.02804055, -0.03489326],
        [-0.0146971 , -0.01844227, -0.02503126, ...,  0.01035003,
         -0.01682558, -0.01603061],
        [-0.03304848, -0.03280932, -0.02324565, ..., -0.00354889,
         -0.00507506, -0.00217535]], shape=(5, 768)),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple ce

In [36]:
# delete document

vector_store.delete(ids = ["77c087a9-e091-4ae9-ad5f-863f42da1717"])

In [37]:
# view document

vector_store.get(include = ['embeddings', 'documents', 'metadatas'])

{'ids': ['aa8b6dcc-38c2-444a-8713-86bc1e89f76e',
  'f8459928-49fb-4bfa-becd-acd57cf17061',
  '297f29a4-ed63-4245-9fc4-0d35113a0d6b',
  'a45a1eb5-135e-432f-b46b-fa19da2cad12'],
 'embeddings': array([[-0.03802421, -0.03025424, -0.00024509, ...,  0.01538167,
         -0.02452593, -0.01501566],
        [-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.03262324,  0.01831091, -0.0045399 , ...,  0.00915938,
         -0.02804055, -0.03489326],
        [-0.0146971 , -0.01844227, -0.02503126, ...,  0.01035003,
         -0.01682558, -0.01603061]], shape=(4, 768)),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league